In [3]:
import sys
sys.path.append("../")

In [4]:
import pandas as pd
import datetime as dt
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from simulation.guru_tester import GuruTester

In [5]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [10]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/H4/{pair}_H4.pkl")
    df_m5 = pd.read_pickle(f"../data/M5/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = RSI(df_an)
    df_an = apply_patterns(df_an)
    df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True
    )
    
    gt.run_test()
    return gt.df_results
    

In [11]:
res = []
for p in ["AUDNZD", "EURUSD", "GBPJPY", "USDCAD" ]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
run_test...
Result: -592.0
prepare_data...
run_test...
Result: -507.0
prepare_data...
run_test...
Result: -552.5
prepare_data...
run_test...
Result: -555.0


In [12]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUDNZD -592.0
EURUSD -507.0
GBPJPY -552.5
USDCAD -555.0


In [13]:
res

[{'pair': 'AUDNZD',
  'res':      running  start_index_m5  profit_factor  loss_factor  start_price  \
  0      False            9312            1.5         -1.0      1.07438   
  1      False           10416            1.5         -1.0      1.07133   
  2      False           12816            1.5         -1.0      1.07154   
  3      False           13152            1.5         -1.0      1.07101   
  4      False           13248            1.5         -1.0      1.06874   
  ..       ...             ...            ...          ...          ...   
  932    False          494591            1.5         -1.0      1.08447   
  933    False          494735            1.5         -1.0      1.08447   
  934    False          495153            1.5         -1.0      1.08294   
  935    False          495297            1.5         -1.0      1.08073   
  936    False          495580            1.5         -1.0      1.08358   
  
       trigger_price  SIGNAL        TP       SL  result            end